In [2]:
from google.cloud import bigquery

In [32]:
client = bigquery.Client(project = "event-trigger-cloud-storage")

In [33]:
table_ref = client.dataset("orders_stg_dataset").table("orders_stg_table")

In [34]:
table_name = client.get_table(table_ref)

In [42]:
full_table_name = str(table_name.full_table_id).replace(':', '.')

In [43]:
full_table_name

'event-trigger-cloud-storage.orders_stg_dataset.orders_stg_table'

In [7]:
existing_schema = table_name.schema

In [8]:
existing_schema

[SchemaField('order_status', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('order_customer_id', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('order_date', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('order_id', 'INTEGER', 'NULLABLE', None, None, (), None)]

In [9]:
new_field = bigquery.SchemaField("source_timestamp", "TIMESTAMP")

In [10]:
new_schema = existing_schema + [new_field]

In [11]:
new_schema

[SchemaField('order_status', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('order_customer_id', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('order_date', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('order_id', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('source_timestamp', 'TIMESTAMP', 'NULLABLE', None, None, (), None)]

In [12]:
table_name.schema = new_schema

In [13]:
table_name.schema

[SchemaField('order_status', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('order_customer_id', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('order_date', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('order_id', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('source_timestamp', 'TIMESTAMP', 'NULLABLE', None, None, (), None)]

In [31]:
new_field = bigquery.SchemaField("source_type", "STRING")


In [32]:
existing_schema = table_name.schema

In [33]:
existing_schema

[SchemaField('order_status', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('order_customer_id', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('order_date', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('order_id', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('source_timestamp', 'TIMESTAMP', 'NULLABLE', None, None, (), None)]

In [34]:
new_schema = existing_schema + [new_field]

In [41]:
new_schema

[SchemaField('order_status', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('order_customer_id', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('order_date', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('order_id', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('source_timestamp', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('source_type', 'STRING', 'NULLABLE', None, None, (), None)]

In [35]:
table_name.schema = new_schema

In [56]:
table_name.id 

AttributeError: 'Table' object has no attribute 'id'

In [38]:
type(table_name)

google.cloud.bigquery.table.Table

In [59]:
table_name.table_id

'orders_stg_table'

In [14]:
client.update_table(table_name, ['schema'])

Table(TableReference(DatasetReference('event-trigger-cloud-storage', 'orders_stg_dataset'), 'orders_stg_table'))

In [15]:
new = [('source_type', 'STRING')]
new_fields = []

In [16]:
for fields in new:
    new_fields.append(bigquery.SchemaField(fields[0], fields[1]))

In [17]:
new_fields

[SchemaField('source_type', 'STRING', 'NULLABLE', None, None, (), None)]

In [18]:
table_name.schema = existing_schema + new_fields

In [21]:
client.update_table(table_name, ['schema'])

NotFound: 404 PATCH https://bigquery.googleapis.com/bigquery/v2/projects/event-trigger-cloud-storage/datasets/orders_stg_dataset/tables/orders_stg_table?prettyPrint=false: Not found: Table event-trigger-cloud-storage:orders_stg_dataset.orders_stg_table

In [22]:
from google.cloud import storage

In [23]:
storage_client = storage.Client(project = 'event-trigger-cloud-storage')

In [25]:
list(storage_client.list_buckets())

[<Bucket: bucket-gcs-to-bq-archival-d>,
 <Bucket: bucket-gcs-to-bq-d>,
 <Bucket: cf-gcs-to-bq-d>,
 <Bucket: event-trigger-cloud-storage-d>,
 <Bucket: gcf-v2-sources-617230007681-asia-south1>]

In [30]:
source_bucket = storage_client.bucket("bucket-gcs-to-bq-d")
destination_bucket = storage_client.bucket("bucket-gcs-to-bq-archival-d")

In [31]:
source_bucket.get_blob("")

'bucket-gcs-to-bq-archival-d'

In [60]:
def move_blobs_from_source_to_destination(source_bucket: str, destination_bucket: str, blob_name: str, project_id: str) -> bool:
    """_summary_

    Args:
        source_bucket (str): _description_
        destination_bucket (str): _description_
        blob_name (str): _description_
        project_id (str): _description_

    Returns:
        bool: _description_
    """
    storage_client          = storage.Client(project = project_id)
    source_bucket           = storage_client.bucket(source_bucket)
    dest_bucket             = storage_client.bucket(destination_bucket)
    source_bucket_name      = source_bucket.name
    destination_bucket_name = dest_bucket.name
    blob                    = source_bucket.get_blob(blob_name)
    blob_copy               = source_bucket.copy_blob(blob, dest_bucket)
    print(
        "Blob {} in bucket {} copied to blob {} in bucket {}.".format(
            blob.name,
            source_bucket.name,
            blob_copy.name,
            dest_bucket.name,
        )
    )
    source_bucket.delete_blob(blob_name.split('/')[1])
    print(
        f"{blob.name} is deleted successfully and moved to archive bucket successfully"
    )
    return True


In [61]:
move_blobs_from_source_to_destination(
                                    source_bucket      = 'bucket-gcs-to-bq-d',
                                    destination_bucket = 'bucket-gcs-to-bq-archival-d',
                                    blob_name          =  r'incoming/part-r-00000-990f5773-9005-49ba-b670-631286032674',
                                    project_id         = 'event-trigger-cloud-storage'
                                )

Blob incoming/part-r-00000-990f5773-9005-49ba-b670-631286032674 in bucket bucket-gcs-to-bq-d copied to blob incoming/part-r-00000-990f5773-9005-49ba-b670-631286032674 in bucket bucket-gcs-to-bq-archival-d.


NotFound: 404 DELETE https://storage.googleapis.com/storage/v1/b/bucket-gcs-to-bq-d/o/part-r-00000-990f5773-9005-49ba-b670-631286032674?prettyPrint=false: No such object: bucket-gcs-to-bq-d/part-r-00000-990f5773-9005-49ba-b670-631286032674

In [48]:
move_blobs_from_source_to_destination(
                                    source_bucket      = 'bucket-gcs-to-bq-d',
                                    destination_bucket = 'bucket-gcs-to-bq-archival-d',
                                    blob_name          =  r'incoming/part-r-00000-990f5773-9005-49ba-b670-631286032674',
                                    project_id         = 'event-trigger-cloud-storage'
                                )

AttributeError: 'str' object has no attribute 'path'